## Getting Started
In this project, you will evaluate the performance and predictive power of a model that has been trained and tested on data collected from homes in suburbs of Boston, Massachusetts. A model trained on this data that is seen as a *good fit* could then be used to make certain predictions about a home — in particular, its monetary value. This model would prove to be invaluable for someone like a real estate agent who could make use of such information on a daily basis.

The dataset for this project originates from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/machine-learning-databases/housing/). The Boston housing data was collected in 1978 and each of the 506 entries represent aggregated data about 14 features for homes from various suburbs in Boston, Massachusetts. For the purposes of this project, the following preprocessing steps have been made to the dataset:
- 16 data points have an `'MEDV'` value of 50.0. These data points likely contain **missing or censored values** and have been removed.
- 1 data point has an `'RM'` value of 8.78. This data point can be considered an **outlier** and has been removed.
- The features `'RM'`, `'LSTAT'`, `'PTRATIO'`, and `'MEDV'` are essential. The remaining **non-relevant features** have been excluded.
- The feature `'MEDV'` has been **multiplicatively scaled** to account for 35 years of market inflation.

Run the code cell below to load the Boston housing dataset, along with a few of the necessary Python libraries required for this project. You will know the dataset loaded successfully if the size of the dataset is reported.

In [ ]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit

#To ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Import supplementary visualizations code visuals.py
import visuals as vs

# Pretty display for notebooks
%matplotlib inline

# Load the Boston housing dataset
data = pd.read_csv('../input/bostonhoustingmlnd/housing.csv')
prices = data['MEDV']
features = data.drop('MEDV', axis = 1)
    
# Success
print("Boston housing dataset has {} data points with {} variables each.".format(*data.shape))

## Data Exploration
In this first section of this project, you will make a cursory investigation about the Boston housing data and provide your observations. Familiarizing yourself with the data through an explorative process is a fundamental practice to help you better understand and justify your results.

Since the main goal of this project is to construct a working model which has the capability of predicting the value of houses, we will need to separate the dataset into **features** and the **target variable**. The **features**, `'RM'`, `'LSTAT'`, and `'PTRATIO'`, give us quantitative information about each data point. The **target variable**, `'MEDV'`, will be the variable we seek to predict. These are stored in `features` and `prices`, respectively.

### Implementation: Calculate Statistics
For your very first coding implementation, you will calculate descriptive statistics about the Boston housing prices. Since `numpy` has already been imported for you, use this library to perform the necessary calculations. These statistics will be extremely important later on to analyze various prediction results from the constructed model.

In the code cell below, you will need to implement the following:
- Calculate the minimum, maximum, mean, median, and standard deviation of `'MEDV'`, which is stored in `prices`.
  - Store each calculation in their respective variable.

In [ ]:
# TODO: Minimum price of the data
minimum_price = np.min(prices)

# TODO: Maximum price of the data
maximum_price = np.max(prices)

# TODO: Mean price of the data
mean_price = np.mean(prices)

# TODO: Median price of the data
median_price = np.median(prices)

# TODO: Standard deviation of prices of the data
std_price = np.std(prices)

# Show the calculated statistics
print("Statistics for Boston housing dataset:\n")
print("Minimum price: ${}".format(minimum_price)) 
print("Maximum price: ${}".format(maximum_price))
print("Mean price: ${}".format(mean_price))
print("Median price ${}".format(median_price))
print("Standard deviation of prices: ${}".format(std_price))

### Question 1 - Feature Observation
As a reminder, we are using three features from the Boston housing dataset: `'RM'`, `'LSTAT'`, and `'PTRATIO'`. For each data point (neighborhood):
- `'RM'` is the average number of rooms among homes in the neighborhood.
- `'LSTAT'` is the percentage of homeowners in the neighborhood considered "lower class" (working poor).
- `'PTRATIO'` is the ratio of students to teachers in primary and secondary schools in the neighborhood.


** Using your intuition, for each of the three features above, do you think that an increase in the value of that feature would lead to an **increase** in the value of `'MEDV'` or a **decrease** in the value of `'MEDV'`? Justify your answer for each.**

**Hint:** This problem can phrased using examples like below.  
* Would you expect a home that has an `'RM'` value(number of rooms) of 6 be worth more or less than a home that has an `'RM'` value of 7?
* Would you expect a neighborhood that has an `'LSTAT'` value(percent of lower class workers) of 15 have home prices be worth more or less than a neighborhood that has an `'LSTAT'` value of 20?
* Would you expect a neighborhood that has an `'PTRATIO'` value(ratio of students to teachers) of 10 have home prices be worth more or less than a neighborhood that has an `'PTRATIO'` value of 15?

**Answer:** As per my intuition, the following would be my analysis:-
* Since, `'RM'` is the average number of rooms among homes in the neighborhood, I believe that  with the increase in the number of rooms in a house the price of the house should also increase.So, with the increase in the value of `'RM'`, the value of `'MEDV'` should also increase and vice-versa. It can be viewed as a directly proportional relationship.
* Generally, if there is a high percentage of lower class homeowners in a neighborhood, the prices of the houses in the neighborhood will be less as poor people can't afford expensive houses. Therefore, with the increase in the value of `'LSAT'`, the value of `'MEDV'` should also decrease and vice-versa. It can be viewed as an inversely proportional relationship.
* According to me the ratio of students to teachers in primary and secondary schools in the neighborhood mustn't be that detrimental to the price of the houses in the neighborhood. There might be some variation in the prices because of `'PTRATO'` but won't be that substantial when compared with the effects of `'RM'` and `'LSAT'`. Hence, `'PTRATIO'` shouldn't have much impact on `'MEDV'` individually, though it may impact the value when coupled with other characteristsics.

----

## Developing a Model
In this second section of the project, you will develop the tools and techniques necessary for a model to make a prediction. Being able to make accurate evaluations of each model's performance through the use of these tools and techniques helps to greatly reinforce the confidence in your predictions.

### Implementation: Define a Performance Metric
It is difficult to measure the quality of a given model without quantifying its performance over training and testing. This is typically done using some type of performance metric, whether it is through calculating some type of error, the goodness of fit, or some other useful measurement. For this project, you will be calculating the [*coefficient of determination*](http://stattrek.com/statistics/dictionary.aspx?definition=coefficient_of_determination), R<sup>2</sup>, to quantify your model's performance. The coefficient of determination for a model is a useful statistic in regression analysis, as it often describes how "good" that model is at making predictions. 

The values for R<sup>2</sup> range from 0 to 1, which captures the percentage of squared correlation between the predicted and actual values of the **target variable**. A model with an R<sup>2</sup> of 0 is no better than a model that always predicts the *mean* of the target variable, whereas a model with an R<sup>2</sup> of 1 perfectly predicts the target variable. Any value between 0 and 1 indicates what percentage of the target variable, using this model, can be explained by the **features**. _A model can be given a negative R<sup>2</sup> as well, which indicates that the model is **arbitrarily worse** than one that always predicts the mean of the target variable._

For the `performance_metric` function in the code cell below, you will need to implement the following:
- Use `r2_score` from `sklearn.metrics` to perform a performance calculation between `y_true` and `y_predict`.
- Assign the performance score to the `score` variable.

In [ ]:
# TODO: Import 'r2_score'
from sklearn.metrics import r2_score

def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    
    # TODO: Calculate the performance score between 'y_true' and 'y_predict'
    score = r2_score(y_true, y_predict)
    
    # Return the score
    return score

### Question 2 - Goodness of Fit
Assume that a dataset contains five data points and a model made the following predictions for the target variable:

| True Value | Prediction |
| :-------------: | :--------: |
| 3.0 | 2.5 |
| -0.5 | 0.0 |
| 2.0 | 2.1 |
| 7.0 | 7.8 |
| 4.2 | 5.3 |

Run the code cell below to use the `performance_metric` function and calculate this model's coefficient of determination.

In [ ]:
# Calculate the performance of this model
score = performance_metric([3, -0.5, 2, 7, 4.2], [2.5, 0.0, 2.1, 7.8, 5.3])
print("Model has a coefficient of determination, R^2, of {:.3f}.".format(score))

* Would you consider this model to have successfully captured the variation of the target variable? 
* Why or why not?

** Hint: **  The R2 score is the proportion of the variance in the dependent variable that is predictable from the independent variable. In other words:
* R2 score of 0 means that the dependent variable cannot be predicted from the independent variable.
* R2 score of 1 means the dependent variable can be predicted from the independent variable.
* R2 score between 0 and 1 indicates the extent to which the dependent variable is predictable. An 
* R2 score of 0.40 means that 40 percent of the variance in Y is predictable from X.

**Answer:** An Rˆ2 score of 0.923 is really good. It means that 92.3 % of the variance in dependent variable(y) can be predicted from the the independent variable(X). Therefore, the model I used for prediction is really good in predicting the values with minimal errors and successfully captures the variation of the target variable.

### Implementation: Shuffle and Split Data
Your next implementation requires that you take the Boston housing dataset and split the data into training and testing subsets. Typically, the data is also shuffled into a random order when creating the training and testing subsets to remove any bias in the ordering of the dataset.

For the code cell below, you will need to implement the following:
- Use `train_test_split` from `sklearn.cross_validation` to shuffle and split the `features` and `prices` data into training and testing sets.
  - Split the data into 80% training and 20% testing.
  - Set the `random_state` for `train_test_split` to a value of your choice. This ensures results are consistent.
- Assign the train and testing splits to `X_train`, `X_test`, `y_train`, and `y_test`.

In [ ]:
# TODO: Import 'train_test_split'
from sklearn.model_selection import train_test_split

# TODO: Shuffle and split the data into training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(features, prices, test_size = 0.2, random_state = 42)

# Success
print("Training and testing split was successful.")

### Question 3 - Training and Testing

* What is the benefit to splitting a dataset into some ratio of training and testing subsets for a learning algorithm?

**Hint:** Think about how overfitting or underfitting is contingent upon how splits on data is done.

**Answer: ** In machine learning we are given a problem to solve for which we can think of a model. The model is trained on the historical data, so that it can identify the critical features through the data and accordingly predict the required variable. Our motivation is to judge performance of the model on out-of-sample data.
* If we use the entire data for training the model, then of course we get a very accurate model that has remembered almost all the features of the training data and will work exceedingly well on the training data. But, it may fail on the testing data as the model is overly complicated and won't be able to generalize properly. This is what causes Overfitting.
* Splitting the out-of-sample data into training and testing data helps us to compute the training and testing accuracy of the model which both need to be good for the model to be used for further prediction. If we oversimplify the model, it will perform bad on both the training and the testing data(Underfitting) and if we overcomplicate it, then it will perform really well on the training data and bad on the testing data(Overfitting). So, splitting the model will help us to find the right fit for the model. Generally, a large chunk of data is used as training data as the model needs to learn from scratch and it's better to provide the model with as much data as possible.

----

## Analyzing Model Performance
In this third section of the project, you'll take a look at several models' learning and testing performances on various subsets of training data. Additionally, you'll investigate one particular algorithm with an increasing `'max_depth'` parameter on the full training set to observe how model complexity affects performance. Graphing your model's performance based on varying criteria can be beneficial in the analysis process, such as visualizing behavior that may not have been apparent from the results alone.

### Learning Curves
The following code cell produces four graphs for a decision tree model with different maximum depths. Each graph visualizes the learning curves of the model for both training and testing as the size of the training set is increased. Note that the shaded region of a learning curve denotes the uncertainty of that curve (measured as the standard deviation). The model is scored on both the training and testing sets using R<sup>2</sup>, the coefficient of determination.  

Run the code cell below and use these graphs to answer the following question.

In [ ]:
# Produce learning curves for varying training set sizes and maximum depths
vs.ModelLearning(features, prices)

### Question 4 - Learning the Data
* Choose one of the graphs above and state the maximum depth for the model. 
* What happens to the score of the training curve as more training points are added? What about the testing curve? 
* Would having more training points benefit the model? 

**Hint:** Are the learning curves converging to particular scores? Generally speaking, the more data you have, the better. But if your training and testing curves are converging with a score above your benchmark threshold, would this be necessary?
Think about the pros and cons of adding more training points based on if the training and testing curves are converging.

**Answer: ** Just keep in mind, the higher the score lesser the errors and the learning curve should be analyzed by taking a mirror image of them from the x-axis.
* `'max_depth = 1'`: It can be noticed that the as the number of training points increases, the training score reduces and the testing score increases and the two scores converge towards a low score, which means high error. Hence, the model is overly simplified and results in high training and testing errors.(Underfit)
* `'max_depth = 3'`: It can be noticed that the as the number of training points increases, the training score reduces a little bit and the testing score increases drastically and the two scores converge towards a high score, which means low error. Hence, the model seems to be a good fit as results in low training and testing errors.(Right fit)
* `'max_depth = 6'`: It can be noticed that the as the number of training points increases, the training score reduces very little(almost negligible) and the testing score increases drastically and the two scores never seem to converge, but move towards a high score, which means low error. Hence, the model seems to be a bit complicated as it results in quite low training and bit high testing errors.(Kind of Overfit)
* `'max_depth = 10'`: It can be noticed that the as the number of training points increases, the training score does not reduce at all and the testing score increases drastically and the two scores never seem to converge, but move towards a high score, which means low error. Hence, the model seems to be overly complicated as it results in completely no training errors and high testing errors.(Overfit)
<br><br>
We'll definitely benefit from having more training points but I feel that we are already able to get a general idea of how the decision tree model would behave with different `'max_depth'` and the given number of training points.

### Complexity Curves
The following code cell produces a graph for a decision tree model that has been trained and validated on the training data using different maximum depths. The graph produces two complexity curves — one for training and one for validation. Similar to the **learning curves**, the shaded regions of both the complexity curves denote the uncertainty in those curves, and the model is scored on both the training and validation sets using the `performance_metric` function.  

** Run the code cell below and use this graph to answer the following two questions Q5 and Q6. **

In [ ]:
vs.ModelComplexity(X_train, y_train)

### Question 5 - Bias-Variance Tradeoff
* When the model is trained with a maximum depth of 1, does the model suffer from high bias or from high variance? 
* How about when the model is trained with a maximum depth of 10? What visual cues in the graph justify your conclusions?

**Hint:** High bias is a sign of underfitting(model is not complex enough to pick up the nuances in the data) and high variance is a sign of overfitting(model is by-hearting the data and cannot generalize well). Think about which model(depth 1 or 10) aligns with which part of the tradeoff.

**Answer: ** As already discussed in Question 4, the following is my analysis on the Bias-Variance tradeoff(The score is the visual cue. The higher the score the lower the errors):-
* `'max_depth = 1'`: This model suffers from high bias(Underfit) as it is overly simplified and is not able to pick up the nuances in the data.
* `'max_depth = 3'`: This model is the right fit as it is neither overly simplified nor overly complicated. It has minimal training as well as testing errors and seems to generalize the data well. Hence, this model is definitely the good fit.
* `'max_depth = 6'`: This model suffers from high variance(Slightly Overfit) as it is a bit complicated and is by-hearting the data because of which it cannot generalize well.
* `'max_depth = 10'`: This model suffers from high variance(Overfit) as it is overly complicated and is by-hearting the data because of which it cannot generalize well.

### Question 6 - Best-Guess Optimal Model
* Which maximum depth do you think results in a model that best generalizes to unseen data? 
* What intuition lead you to this answer?

** Hint: ** Look at the graph above Question 5 and see where the validation scores lie for the various depths that have been assigned to the model. Does it get better with increased depth? At what point do we get our best validation score without overcomplicating our model? And remember, Occams Razor states "Among competing hypotheses, the one with the fewest assumptions should be selected."

**Answer: ** By looking at the complexity curve, it can be rightly said that `'max_depth = 4'` is the maximum depth that best generalizes to the unseen data as it produces the best fit of training and validation scores. As we reduce the maximum depth the model becomes Underfit and as we increase the maximum depth the model becomes Overfit. The validation score is at the max at max_depth = 4 and also the training score is pretty high and there is not a significant difference between both the scores.

-----

## Evaluating Model Performance
In this final section of the project, you will construct a model and make a prediction on the client's feature set using an optimized model from `fit_model`.

### Question 7 - Grid Search
* What is the grid search technique?
* How it can be applied to optimize a learning algorithm?

** Hint: ** When explaining the Grid Search technique, be sure to touch upon why it is used,  what the 'grid' entails and what the end goal of this method is. To solidify your answer, you can also give an example of a parameter in a model that can be optimized using this approach.

**Answer: ** Grid search technique is a type of hyperparameter optimization technique. A hyperparameter is a parameter of the model which is set before the training process starts.
<br>
>The steps involved in any machine learning problem are as follows:-
1. A bunch of models are trained using the training data.
2. The best model is picked using the cross-validation data.
3. The chosen model is then provided with the test data to check if the model is actually good.
<br>

So, the second step where fine-tuning of the model using the cross-validation data is involved, grid search comes into play. In order to optimize a learning algorithm using grid search, a dictionary with hyperparameters as the keys and all their possible values as values and a metric(f1_score, r2_score, e.t.c.) as scorer are provided.<br>
<b>Example</b><br>Lets assume we are doing a grid search on a Logistic regression model. In this case, the degree of the polynomial with be the hyperparameter and lets take f1_score as the evaluation metric. So, a dictionary like {'degree': [1,2,3,4]} and f1_score as the scorer will be provided for grid search.


### Question 8 - Cross-Validation

* What is the k-fold cross-validation training technique? 

* What benefit does this technique provide for grid search when optimizing a model?

**Hint:** When explaining the k-fold cross validation technique, be sure to touch upon what 'k' is, how the dataset is split into different parts for training and testing and the number of times it is run based on the 'k' value.

When thinking about how k-fold cross validation helps grid search, think about the main drawbacks of grid search which are hinged upon **using a particular subset of data for training or testing** and how k-fold cv could help alleviate that. You can refer to the [docs](http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation) for your answer.

**Answer: ** K-fold cross-validation is a training technique in which:-
* Entire data is split into k branches.
* 1 branch is used as a testing set and the remaining k-1 branches are used as testing set. This same process is carried out k times.
* the accuracy of the out-of-sample performance of the model is calculated as the average of the accuracy of all the k tests.
<br><br>
The k-fold cross-validation training technique is much robust when compared to the train-test split method as k-fold cross validation is ran k times and the performance is computed on the average of all the k iterations. Hence, it reduces the variance of the single test-test split. It's drawback is that it does not work that well on non-uniformly distributed data.
<br><br>
If a grid search is performed on a train-test split then there is a high possiblity that the model is overly generalised and does not perform well on new data. But, a k-fold cv will help to alleviate this problem of overfitting as accuracy of the model is computed as the average of accuracy of k iterations, thereby reducing the variance.

### Implementation: Fitting a Model
Your final implementation requires that you bring everything together and train a model using the **decision tree algorithm**. To ensure that you are producing an optimized model, you will train the model using the grid search technique to optimize the `'max_depth'` parameter for the decision tree. The `'max_depth'` parameter can be thought of as how many questions the decision tree algorithm is allowed to ask about the data before making a prediction. Decision trees are part of a class of algorithms called *supervised learning algorithms*.

In addition, you will find your implementation is using `ShuffleSplit()` for an alternative form of cross-validation (see the `'cv_sets'` variable). While it is not the K-Fold cross-validation technique you describe in **Question 8**, this type of cross-validation technique is just as useful!. The `ShuffleSplit()` implementation below will create 10 (`'n_splits'`) shuffled sets, and for each shuffle, 20% (`'test_size'`) of the data will be used as the *validation set*. While you're working on your implementation, think about the contrasts and similarities it has to the K-fold cross-validation technique.

Please note that ShuffleSplit has different parameters in scikit-learn versions 0.17 and 0.18.
For the `fit_model` function in the code cell below, you will need to implement the following:
- Use [`DecisionTreeRegressor`](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) from `sklearn.tree` to create a decision tree regressor object.
  - Assign this object to the `'regressor'` variable.
- Create a dictionary for `'max_depth'` with the values from 1 to 10, and assign this to the `'params'` variable.
- Use [`make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html) from `sklearn.metrics` to create a scoring function object.
  - Pass the `performance_metric` function as a parameter to the object.
  - Assign this scoring function to the `'scoring_fnc'` variable.
- Use [`GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) from `sklearn.grid_search` to create a grid search object.
  - Pass the variables `'regressor'`, `'params'`, `'scoring_fnc'`, and `'cv_sets'` as parameters to the object. 
  - Assign the `GridSearchCV` object to the `'grid'` variable.

In [ ]:
# TODO: Import 'make_scorer', 'DecisionTreeRegressor', and 'GridSearchCV'
from sklearn.metrics import make_scorer
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    # sklearn version 0.18: ShuffleSplit(n_splits=10, test_size=0.1, train_size=None, random_state=None)
    # sklearn versiin 0.17: ShuffleSplit(n, n_iter=10, test_size=0.1, train_size=None, random_state=None)
    cv_sets = ShuffleSplit(n_splits = 10, test_size = 0.20, random_state = 0)

    # TODO: Create a decision tree regressor object
    regressor = DecisionTreeRegressor()

    # TODO: Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = {'max_depth': np.arange(1,11)}

    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(r2_score)

    # TODO: Create the grid search cv object --> GridSearchCV()
    # Make sure to include the right parameters in the object:
    # (estimator, param_grid, scoring, cv) which have values 'regressor', 'params', 'scoring_fnc', and 'cv_sets' respectively.
    grid = GridSearchCV(regressor, params, scoring = scoring_fnc, cv = cv_sets)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

### Making Predictions
Once a model has been trained on a given set of data, it can now be used to make predictions on new sets of input data. In the case of a *decision tree regressor*, the model has learned *what the best questions to ask about the input data are*, and can respond with a prediction for the **target variable**. You can use these predictions to gain information about data where the value of the target variable is unknown — such as data the model was not trained on.

### Question 9 - Optimal Model

* What maximum depth does the optimal model have? How does this result compare to your guess in **Question 6**?  

Run the code block below to fit the decision tree regressor to the training data and produce an optimal model.

In [ ]:
# Fit the training data to the model using grid search
reg = fit_model(X_train, y_train)

# Produce the value for 'max_depth'
print("Parameter 'max_depth' is {} for the optimal model.".format(reg.get_params()['max_depth']))

** Hint: ** The answer comes from the output of the code snipped above.

**Answer: ** As per the code, the optimal model must have a max_depth of 4 which is exactly similar to the score that we computed through the complexity curve of the model. So, using both the techniques: Complexity curve and Grid search, we came up with the same result. Hence, our model should perform well on the testing data.

### Question 10 - Predicting Selling Prices
Imagine that you were a real estate agent in the Boston area looking to use this model to help price homes owned by your clients that they wish to sell. You have collected the following information from three of your clients:

| Feature | Client 1 | Client 2 | Client 3 |
| :---: | :---: | :---: | :---: |
| Total number of rooms in home | 5 rooms | 4 rooms | 8 rooms |
| Neighborhood poverty level (as %) | 17% | 32% | 3% |
| Student-teacher ratio of nearby schools | 15-to-1 | 22-to-1 | 12-to-1 |

* What price would you recommend each client sell his/her home at? 
* Do these prices seem reasonable given the values for the respective features? 

**Hint:** Use the statistics you calculated in the **Data Exploration** section to help justify your response.  Of the three clients, client 3 has has the biggest house, in the best public school neighborhood with the lowest poverty level; while client 2 has the smallest house, in a neighborhood with a relatively high poverty rate and not the best public schools.

Run the code block below to have your optimized model make predictions for each client's home.

In [ ]:
# Produce a matrix for client data
client_data = [[5, 17, 15], # Client 1
               [4, 32, 22], # Client 2
               [8, 3, 12]]  # Client 3

# Show predictions
for i, price in enumerate(reg.predict(client_data)):
    print("Predicted selling price for Client {}'s home: ${:,.2f}".format(i+1, price))

**Answer: ** The recommended prices of each of the clients are listed above as the output of the code.
#### Visualization
Below are the plots showing the variation of MEDV with respect to RM, LSTAT and PTRATIO.

In [ ]:
from matplotlib import pyplot as plt

clients = np.transpose(client_data)
pred = reg.predict(client_data)
for i, feature in enumerate(['RM','LSTAT','PTRATIO']):
    plt.scatter(features[feature], prices, alpha=0.2, c='blue')
    plt.scatter(clients[i], pred, c='black', marker='s')
    plt.xlabel(feature)
    plt.ylabel('MEDV')
    plt.show()

The following is my analysis on the reasonability of the predicted prices:-
* `'Client 1'` is living in a home with 5 rooms,  17% neighborhood poverty level and 15:1 student to teacher ratio. The home is having less rooms than the average number of rooms, a slightly higher neighbourhood poverty level than the mean poverty level and less than average student to teacher ratio. This implies that it is a small house in a middle class locality less accessible from best public schools. Therefore, the price of the house should be somewhere below the average and the value is predicted correctly.
* `'Client 2'` is living in a home with 4 rooms,  32% neighborhood poverty level and 22:1 student to teacher ratio. The home is having least number of rooms, a very high neighbourhood poverty and the worst student to teacher ratio. This implies that it is a very small house in a poor locality with very bad accessibility from best public schools. Therefore, the price of the should be close to the lowest price and the value seems to be predicted correctly.
* `'Client 3'` is living in a home with 8 rooms,  3% neighborhood poverty level and 12:1 student to teacher ratio. The home is having most number of rooms , one of the lowest neighbourhood poverty level and the best student to teacher ratio. This implies that it is a very big house in a high class locality with great accessibility to the best public schools. Therefore, the price of the should be somewhere close to the highest price and the value seems to be predicted correctly.

### Sensitivity
An optimal model is not necessarily a robust model. Sometimes, a model is either too complex or too simple to sufficiently generalize to new data. Sometimes, a model could use a learning algorithm that is not appropriate for the structure of the data given. Other times, the data itself could be too noisy or contain too few samples to allow a model to adequately capture the target variable — i.e., the model is underfitted. 

**Run the code cell below to run the `fit_model` function ten times with different training and testing sets to see how the prediction for a specific client changes with respect to the data it's trained on.**

In [ ]:
vs.PredictTrials(features, prices, fit_model, client_data)

### Question 11 - Applicability

* In a few sentences, discuss whether the constructed model should or should not be used in a real-world setting.  

**Hint:** Take a look at the range in prices as calculated in the code snippet above. Some questions to answering:
- How relevant today is data that was collected from 1978? How important is inflation?
- Are the features present in the data sufficient to describe a home? Do you think factors like quality of apppliances in the home, square feet of the plot area, presence of pool or not etc should factor in?
- Is the model robust enough to make consistent predictions?
- Would data collected in an urban city like Boston be applicable in a rural city?
- Is it fair to judge the price of an individual home based on the characteristics of the entire neighborhood?

**Answer: **
* The data collected from 1978 is definitely not that relevant as a lot as changed since then including the demographics, economic strata, inflation rate, per capita income and many more things.
<br>
* Currently, there are only three features present in the data, which is definitely not enough to describe a home. Factors such as total income, number of family members, crime rate, distances from public amenities, quality of home appliances, square feet of the plot area, e.t.c. should also factor in. With all these we would be able to create a very accurate model.
<br>
* With the given set of features it can be said that the model is robust enough and predicts prices with a less margin of error.
<br>
* No, the data collected in an urban city like Boston will not be applicable in a rural city as this would be like comparing apples with oranges. The demographics, economic status, standards of living, requirements and many other major factors are different for a rural city when compared with a city like Boston.
<br>
* Mostly, the price of an individual home can be judged based on the characteristics of an entire neighborhood as all the houses in a particular neighborhood are more or less the similar. There can of course be very well built and an expensive house in a poor locality or vice-versa but inconsistencies are always there and these can be neglected when generalizing.